## Problem One: Confirmation of competition contents

* What to learn and what to predict?

* What kind of file to create and submit to Kaggle?

* What kind of index value will be used to evaluate the submissions?





What to learn and what to predict
• To predict how capable each applicant is of repaying a loan
What kind of file to create and submit to Kaggle?
• A file predicting the probability for the TARGET variable. And should contain a header of both SK_ID_CURR and TARGET
What kind of index value will be used to evaluate the submissions?
• Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target. The ROC curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings. The true-positive rate is also known as sensitivity, recall or probability of detection[8] in machine learning.

## Problem Two: Learning and verification

In [142]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [143]:
# Loading the dataset
df_train = pd.read_csv("application_train.csv")
df_test = pd.read_csv("application_test.csv")

df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
display(df_train.info())
display(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB


None

In [145]:
display(df_train.shape)
display(df_test.shape)

(307511, 122)

(48744, 121)

* The test dataset is missing the target column because target is the variable we're trying to predict.
* We have 16 columns of data type object

In [146]:
(df_train.isna().sum() > 0).value_counts()

True     67
False    55
dtype: int64

In [147]:
df_test.columns[(df_test.isna().sum() > 0)]

Index(['AMT_ANNUITY', 'NAME_TYPE_SUITE', 'OWN_CAR_AGE', 'OCCUPATION_TYPE',
       'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG',
       'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG',
       'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG',
       'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
       'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG',
       'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE',
       'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE',
       'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE',
       'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI',
       'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI',
       'LANDAREA_MEDI', 'LIVINGAPART

In [148]:
# Function to check the correlation between a quantitative variable and the target
def compare_corr(features):
    features.append("TARGET")
    return df_train[features].corr()


def compute_chi(cat_variable):
    """
    Function to compute the correlation of a categorical variable and the target
    """
    # Create a contingency table
    contingency_table = pd.crosstab(df_train[cat_variable], df_train["TARGET"])

    # Calculate the chi-square statistic and p-value
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Print the results
    print("Chi-square statistic:", chi2)
    print("p-value:", p)

In [149]:
# Getting the percentage of missing values in the train dataset

percent_missing_train = df_train.isna().sum() / len(df_train) * 100

col_missing_train = df_train.columns[(percent_missing_train != 0)]

percent_missing_train[percent_missing_train > 0].sort_values(ascending=False).tail(60)

LIVINGAPARTMENTS_MODE           68.354953
LIVINGAPARTMENTS_MEDI           68.354953
LIVINGAPARTMENTS_AVG            68.354953
FLOORSMIN_MODE                  67.848630
FLOORSMIN_MEDI                  67.848630
FLOORSMIN_AVG                   67.848630
YEARS_BUILD_MODE                66.497784
YEARS_BUILD_MEDI                66.497784
YEARS_BUILD_AVG                 66.497784
OWN_CAR_AGE                     65.990810
LANDAREA_AVG                    59.376738
LANDAREA_MEDI                   59.376738
LANDAREA_MODE                   59.376738
BASEMENTAREA_MEDI               58.515956
BASEMENTAREA_AVG                58.515956
BASEMENTAREA_MODE               58.515956
EXT_SOURCE_1                    56.381073
NONLIVINGAREA_MEDI              55.179164
NONLIVINGAREA_MODE              55.179164
NONLIVINGAREA_AVG               55.179164
ELEVATORS_MEDI                  53.295980
ELEVATORS_MODE                  53.295980
ELEVATORS_AVG                   53.295980
WALLSMATERIAL_MODE              50

Most of the features with missing values are related with the building or home of the client, we will investigate their individual correlation with the target later on. For now, my hypothesis is that there should be some relationship between the OCCUPATION_TYPE feature and the rate of default, so we shall test this theory in order to decide if to bother with imputation of missing variables for OCCUPATION_TYPE.

In [150]:
# Checking if there's any relationship between OCCUPATION_TYPE and rate of repayment
(1 - df_train.groupby("OCCUPATION_TYPE").mean().round(2)["TARGET"]).sort_values(ascending=False)

OCCUPATION_TYPE
Accountants              0.95
Core staff               0.94
HR staff                 0.94
High skill tech staff    0.94
IT staff                 0.94
Managers                 0.94
Secretaries              0.93
Medicine staff           0.93
Private service staff    0.93
Realty agents            0.92
Sales staff              0.90
Cooking staff            0.90
Cleaning staff           0.90
Security staff           0.89
Waiters/barmen staff     0.89
Laborers                 0.89
Drivers                  0.89
Low-skill Laborers       0.83
Name: TARGET, dtype: float64

We can see that there is some change in the data as we move from corporate jobs to manual jobs, this information might be useful for our model, so we shall try to fill in the missing values OCCUPATION_TYPE and further investigate if it has any impact on our model.

In [151]:
# Checking the rate of repayment for clients with missing occupation types
1 - df_train[df_train.OCCUPATION_TYPE.isna()][["TARGET"]].mean()

TARGET    0.934869
dtype: float64

In [152]:
compute_chi("OCCUPATION_TYPE")

Chi-square statistic: 1402.8467961927515
p-value: 3.7844998567642684e-288


In [153]:
# replacing the missing values in OCCUPATION_TYPE with the most frequent observation
most_frequent_occupation = df_train["OCCUPATION_TYPE"].mode().iloc[0]

# Replace missing values with the most frequent occupation
df_train["OCCUPATION_TYPE"].fillna(most_frequent_occupation, inplace=True)
df_test["OCCUPATION_TYPE"].fillna(df_test["OCCUPATION_TYPE"].mode().iloc[0], inplace=True)

In [154]:
# Checking if the imputation drastically affects the correlation between OCCUPATION_TYPE and the target 
compute_chi("OCCUPATION_TYPE")

Chi-square statistic: 1193.3947394399675
p-value: 3.4203518801583733e-243


There is only a slight decrease in correlation between the two variables after mode imputation so we shall accept this imputation technique and also keep OCCUPATION_TYPE.

In [155]:
# Getting and displaying the correlation of all missing variables and the target
missing_val_corr = compare_corr(percent_missing_train[percent_missing_train != 0].index.to_list())

# We take the absolute values of the correlation coefficients to account for negative correlation
missing_val_corr.TARGET.apply(lambda x: abs(x)).sort_values(ascending=False).head(60)

TARGET                          1.000000
EXT_SOURCE_3                    0.178919
EXT_SOURCE_2                    0.160472
EXT_SOURCE_1                    0.155317
DAYS_LAST_PHONE_CHANGE          0.055218
FLOORSMAX_AVG                   0.044003
FLOORSMAX_MEDI                  0.043768
FLOORSMAX_MODE                  0.043226
AMT_GOODS_PRICE                 0.039645
OWN_CAR_AGE                     0.037612
ELEVATORS_AVG                   0.034199
ELEVATORS_MEDI                  0.033863
FLOORSMIN_AVG                   0.033614
FLOORSMIN_MEDI                  0.033394
LIVINGAREA_AVG                  0.032997
LIVINGAREA_MEDI                 0.032739
FLOORSMIN_MODE                  0.032698
TOTALAREA_MODE                  0.032596
DEF_30_CNT_SOCIAL_CIRCLE        0.032248
ELEVATORS_MODE                  0.032131
DEF_60_CNT_SOCIAL_CIRCLE        0.031276
LIVINGAREA_MODE                 0.030685
APARTMENTS_AVG                  0.029498
APARTMENTS_MEDI                 0.029184
APARTMENTS_MODE 

Most of the features have very poor correlation power. We shall only consider EXT_SOURCE_3, EXT_SOURCE_2, and EXT_SOURCE_1 because although they have poor correlation with the target, their correlation coefficients surpass all other features, and domain knowledge suggests that external sources play a huge role in credit allocation.

**Feature Engineering Approach:** We shall take the mean the values of all three variables and see how that affects the predictive power of our model.

In [156]:
# Handling missing data in EXT_SOURCE_3, EXT_SOURCE_2, and EXT_SOURCE_1

df_train["EXT_SOURCE_MEAN"] = df_train[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis=1)
df_test["EXT_SOURCE_MEAN"] = df_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis=1)

df_train[["EXT_SOURCE_MEAN", "TARGET"]].corr()

,EXT_SOURCE_MEAN,TARGET
EXT_SOURCE_MEAN,1.000000,-0.222052
TARGET,-0.222052,1.000000


The result of the correlation between EXT_SOURCE_MEAN, which is the mean of EXT_SOURCE_1, EXT_SOURCE_2, and EXT_SOURCE_3, and TARGET show an inverse increament when compared to the correlation between TARGET and each individual feature.

In [157]:
features_to_drop = [col for col in col_missing_train if col != "OCCUPATION_TYPE"]
len(features_to_drop)

66

In [158]:
len(df_train.columns)

123

In [159]:
# dropping features
df_train.drop(features_to_drop, axis=1, inplace=True)
df_test.drop(features_to_drop, axis=1, inplace=True)
percent_missing_train.drop(features_to_drop, inplace=True)
percent_missing_train

SK_ID_CURR                      0.000000
TARGET                          0.000000
NAME_CONTRACT_TYPE              0.000000
CODE_GENDER                     0.000000
FLAG_OWN_CAR                    0.000000
FLAG_OWN_REALTY                 0.000000
CNT_CHILDREN                    0.000000
AMT_INCOME_TOTAL                0.000000
AMT_CREDIT                      0.000000
NAME_INCOME_TYPE                0.000000
NAME_EDUCATION_TYPE             0.000000
NAME_FAMILY_STATUS              0.000000
NAME_HOUSING_TYPE               0.000000
REGION_POPULATION_RELATIVE      0.000000
DAYS_BIRTH                      0.000000
DAYS_EMPLOYED                   0.000000
DAYS_REGISTRATION               0.000000
DAYS_ID_PUBLISH                 0.000000
FLAG_MOBIL                      0.000000
FLAG_EMP_PHONE                  0.000000
FLAG_WORK_PHONE                 0.000000
FLAG_CONT_MOBILE                0.000000
FLAG_PHONE                      0.000000
FLAG_EMAIL                      0.000000
OCCUPATION_TYPE 

In [160]:
print(len(df_train.columns), len(df_test.columns))

57 56


Train dataset still has its target column, we shall extract that in a bit.

In [161]:
# Splitting and Training for Feature Selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import LabelEncoder


# Instantiate a LabelEncoder object
le = LabelEncoder()

obj_cols = df_train.columns[df_train.dtypes == "object"]

# Encoding all features of object type 
for i in obj_cols:
    df_train[i] = le.fit_transform(df_train[i])
    df_test[i] = le.fit_transform(df_test[i])
    
# Checking that all object data type have been encoded
df_train.select_dtypes("object").columns

Index([], dtype='object')

We've succeeded in filling the missing values and encoding the columns of data type object.

In [165]:
df_train.columns[df_train.isna().sum() > 0]

Index(['EXT_SOURCE_MEAN'], dtype='object')

In [166]:
# Replacing the few missing values in EXT_SOURCE_MEAN with the entire feature's mean
df_train.EXT_SOURCE_MEAN.fillna(df_train["EXT_SOURCE_MEAN"].mean(), inplace=True)

In [167]:
# Confirming that there is no missing values in the dataset
df_train.columns[df_train.isna().sum() > 0]

Index([], dtype='object')

### Training and Evaluation of Test Data

In [171]:
X = df_train.drop("TARGET", axis=1)
y = df_train["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.fit_transform(X_test)

dtree = DecisionTreeClassifier()
dtree.fit(X_train_transformed, y_train)
dtree.

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,EXT_SOURCE_MEAN
0,100002,0,1,0,1,0,202500.0,406597.5,7,4,...,0,0,0,0,0,0,0,0,0,0.161787
1,100003,0,0,0,0,0,270000.0,1293502.5,4,1,...,0,0,0,0,0,0,0,0,0,0.466757
2,100004,1,1,1,1,0,67500.0,135000.0,7,4,...,0,0,0,0,0,0,0,0,0,0.642739
3,100006,0,0,0,1,0,135000.0,312682.5,7,4,...,0,0,0,0,0,0,0,0,0,0.650442
4,100007,0,1,0,1,0,121500.0,513000.0,7,4,...,0,0,0,0,0,0,0,0,0,0.322738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,1,0,0,0,157500.0,254700.0,7,4,...,0,0,0,0,0,0,0,0,0,0.413601
307507,456252,0,0,0,1,0,72000.0,269550.0,3,4,...,0,0,0,0,0,0,0,0,0,0.115992
307508,456253,0,0,0,1,0,153000.0,677664.0,7,1,...,0,0,0,0,0,0,0,0,0,0.499536
307509,456254,0,0,0,1,0,171000.0,370107.0,1,4,...,0,0,0,0,0,0,0,0,0,0.587593


## Problem Three: Estimation on test data

## Problem Four: Feature engineering